# All CMIP6 Phase 2 GCM 2070-2100

Notebook environment to migrate NetCDF files to CF compliant zarr

In [5]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [6]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
#P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards\01_Present\Extreme_Wave_Height
# Define (local and) remote drives
# gca_data_dir = pathlib.Path.home().joinpath(r"OneDrive - Stichting Deltares\Documents\GitHub")
gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas", r"MSc_students\ClenmarRowe\Data\All_Datasets", r"Orig_Datasets\01_Hazards\02_Future")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [7]:
GCM_all=["MRI-ESM2-0","MPI-ESM1-2-LR","KIOST-ESM","IPSL-CM6A-LR","CMCC-CM2-SR5","AWI-CM-1-1-MR"]

<IPython.core.display.Javascript object>

In [12]:
for i,GCM in tqdm(enumerate(GCM_all),desc="GCM completed"):# Project paths & files (manual input)
    if i == 0:
        continue
    dataset_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM)
    os.mkdir(dataset_dir)
    dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM,GCM+"_WW3_3Hourly_2071-2100.nc")
    CF_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM,"CF")  # directory to save output CF check files
    
    
    #For SSP1-26

    orig_dataset_dir= gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100")
    # Specify the path to the directory containing the netCDF files
    dir="SSP1-26"

    # Get a list of all netCDF files in the specified directory
    files = sorted(glob.glob(os.path.join(orig_dataset_dir , dir,  "*.nc")))

    datasets=[]
    for file_name in files:
        if GCM in file_name:
            # Open datasets using lazily in Chunks
            datasets.append(xr.open_dataset(file_name, chunks={'time':  1}))
    datasets


    # Concatenate the datasets along the time dimension
    merged_SSP1_RCP26 = xr.concat(datasets, dim='time')#.compute()

    # Save the merged dataset to a new netCDF file in the current directory
    # merged.to_netcdf('merged_output.nc')
    merged_SSP1_RCP26
    #For SSP5-85

    # Specify the path to the directory containing the netCDF files
    dir="SSP5-85"

    # Get a list of all netCDF files in the specified directory
    files = sorted(glob.glob(os.path.join(orig_dataset_dir , dir,  "*.nc")))

    datasets=[]
    for file_name in files:
        if GCM in file_name:
            # Open datasets using lazily in Chunks
            datasets.append(xr.open_dataset(file_name, chunks={'time':  1}))


    # Concatenate the datasets along the time dimension
    merged_SSP5_RCP85 = xr.concat(datasets, dim='time')

    # Save the merged dataset to a new netCDF file in the current directory
    # merged.to_netcdf('merged_output.nc')
    merged_SSP5_RCP85
    combined_ds= merged_SSP5_RCP85
    combined_ds=combined_ds.rename({'hs': 'hs_5_85_'+GCM.replace("-","_")})

    combined_ds["hs_1_26_"+GCM.replace("-","_")]=merged_SSP1_RCP26["hs"]
    combined_ds=combined_ds.drop_vars("MAPSTA")
    combined_ds
    ds=combined_ds
    ### Check CF compliancy original NetCDF files
    # %%capture cap --no-stderr
    # # check original CF compliancy (for first file)

    # check_compliancy(testfile=files[0], 
    #                 working_dir=CF_dir
    #                 )
    # # # save original CF compliancy (for first file)
    # save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)
    # ### Make CF compliant alterations to the NetCDF files (dataset dependent)
    import json

    # NetCDF attribute alterations by means of metadata template
    metadata_json=orig_dataset_dir.joinpath("metadata_"+GCM+"_2071-2100.json")

    f_global = open(metadata_json)
    meta_global = json.load(f_global)


    for attr_name, attr_val in meta_global.items():
        if attr_name == 'PROVIDERS':
            attr_val = json.dumps(attr_val)
        ds.attrs[attr_name] = attr_val

    ds.attrs['Conventions'] = "CF-1.8"
    ds
    # # Slice to reduce size of CF checked NetCDF
    # ds_slice=ds.sel(time=slice('2065-01-01','2065-01-02'))
    # ds_slice
    # # var=ds['swh'].isel(time=0)
    # # var.plot()
    # # plt.show()
    # # Write the xarray dataset to a netCDF file
    # #Compliant netcdf
    # dataset_dir_path_CF=pathlib.Path().joinpath(str(dataset_dir_path)[0:-3]+"_CF"+".nc")


    # # ds.to_netcdf(path=dataset_dir_path_CF)
    # #ds_slice.to_netcdf(path=r"C:\Users\rowe\OneDrive - Stichting Deltares\Desktop\delete\era5_coastal_Hs_all_years_CF.nc")
    # ds_slice.to_netcdf(path=dataset_dir_path_CF)
    # ### Check CF compliancy modified NetCDF files
    # # open datasets (only first file, rest is the same)
    # ds_slice = xr.open_dataset(dataset_dir_path_CF)

    # # check original dataset
    # ds_slice
    # %%capture cap --no-stderr
    # # check original CF compliancy (for first file)

    # check_compliancy(testfile=dataset_dir_path_CF, 
    #                 working_dir=CF_dir
    #                 )
    # # save original CF compliancy (for first file)
    # save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)
    ### Write data to Zarr files
   


    # # export to zarr in write mode (to overwrite if exists)

    with ProgressBar():
        ds.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")

    if i == 0:
        combined_all = ds
    else:
        combined_all['hs_5_85_' + GCM.replace("-", "_")] = ds['hs_5_85_' + GCM.replace("-", "_")]
        combined_all['hs_1_26_' + GCM.replace("-", "_")] = ds['hs_1_26_' + GCM.replace("-", "_")]
combined_all
    
# # export to zarr in write mode (to overwrite if exists)
# ds.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")

GCM completed: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 3hr 52m


GCM completed: 2it [3:56:50, 7105.42s/it]

[########################################] | 100% Completed | 3hr 43m


c:\Users\rowe\AppData\Local\miniconda3\envs\coclico\Lib\site-packages\xarray\core\indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
c:\Users\rowe\AppData\Local\miniconda3\envs\coclico\Lib\site-packages\xarray\core\indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return se

...

GCM completed: 5it [15:52:20, 12697.51s/it]

[########################################] | 100% Completed | 3hr 52m


GCM completed: 6it [19:48:48, 11888.16s/it]


<xarray.Dataset>
Dimensions:                (longitude: 720, latitude: 361, time: 87656)
Coordinates:
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.207101_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2071_2100_CMIP6_MR...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [14]:
# combined_all = ds
metadata_json=orig_dataset_dir.joinpath("metadata_All_Phase2_2071-2100.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    combined_all.attrs[attr_name] = attr_val

combined_all.attrs['Conventions'] = "CF-1.8"
combined_all

<xarray.Dataset>
Dimensions:                (longitude: 720, latitude: 361, time: 87656)
Coordinates:
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.207101_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2071_2100_CMIP6_MR...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [15]:
comb_dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100","Combined_CMIP_P2_WW3_3Hourly_2071-2100.nc")
dataset_dir_path_CF=pathlib.Path().joinpath(str(comb_dataset_dir_path)[0:-3]+"_CF"+".nc")
combined_all.isel(time=0).to_netcdf(path=dataset_dir_path_CF)

%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                working_dir=CF_dir
                )
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [16]:
combined_all

<xarray.Dataset>
Dimensions:                (longitude: 720, latitude: 361, time: 87656)
Coordinates:
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.207101_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2071_2100_CMIP6_MR...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [19]:
with ProgressBar():
    combined_all_rechunk = combined_all.chunk({'time': 1, 'latitude': 361, 'longitude': 720})   
combined_all_rechunk

<xarray.Dataset>
Dimensions:                (longitude: 720, latitude: 361, time: 87656)
Coordinates:
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    WAVEWATCH_III_version_number:  6.07
    WAVEWATCH_III_switches:        F90 NOGRB NC4 SCRIP SHRD PR3 UQ FLX4 LN1 S...
    product_name:                  ww3.207101_hs.nc
    area:                          Global regular grid output
    latitude_resolution:           0.
    longitude_resolution:          0.
    ...                            ...
    DOI:                           
    LONG_NAME:                     SIGNIFICANT_WAVE_HEIGHT_2071_2100_CMIP6_MR...
    UNITS:                         meters
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8

<IPython.core.display.Javascript object>

In [20]:
with ProgressBar():
    combined_all_rechunk.to_zarr(str(comb_dataset_dir_path).replace(".nc", ".zarr"), mode="w")

[                                        ] | 0% Completed | 3hr 38mss


KeyboardInterrupt: 

<IPython.core.display.Javascript object>